# 事前準備

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# コンペティションデータ準備
df_train = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")
df_test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")

# 米国特許商標庁の分野カテゴリーデータを準備
cpc_codes = pd.read_csv("../input/cpc-codes/titles.csv")

In [ ]:
# 訓練データ表示
df_train.head()

In [ ]:
# 分野カテゴリーデータ表示
cpc_codes.head()

In [ ]:
# 訓練データと分野カテゴリーデータを結合
cpc_codes = cpc_codes.rename(columns = {"code" : "context"})
df_train = pd.merge(df_train, cpc_codes[["context","title","section","class"]], on ="context", how = "left")
df_test = pd.merge(df_test, cpc_codes[["context","title","section","class"]], on ="context", how = "left")
df_train.head()

# 訓練データ

In [ ]:
# 要約情報表示
df_train.info()

総行数：36473\
欠損行はなし\
score列はfloat型

In [ ]:
# csectionごとにscoreの統計量表示
df_train.groupby("section").describe()["score"]

### 各セクションの意味は以下の通り

A: Human Necessities(人間の必需品)\
B: Operations and Transport(運用と輸送)\
C: Chemistry and Metallurgy(化学および冶金学)\
D: Textiles(テキスタイル)\
E: Fixed Constructions(固定構造)\
F: Mechanical Engineering(機械工学)\
G: Physics(物理学)\
H: Electricity(電気)\
Y: Emerging Cross-Sectional Technologies(新しい横断的テクノロジー)\

In [ ]:
# scoreごとの集計
plt.bar(df_train["score"].value_counts().sort_values(ascending=True).index,
        df_train["score"].value_counts().sort_values(ascending=True),
        tick_label = df_train["score"].value_counts().sort_values(ascending=True).index,width=0.2)

In [ ]:
# sectionごとの行数を表示
label=['A','B','C','D','E','F','G','H']
plt.bar(df_train["section"].value_counts().sort_index().index,df_train["section"].value_counts().sort_index(),tick_label =label)


### 統計量を確認した結果
どのsectionにおいてもscoreの平均値は0.35付近、標準偏差は0.25付近と大差ない。\
ただしsectionによってデータ数は大きく異なる（Bのデータ数はDの概ね7倍のデータ量）

In [ ]:
# 全てのcontextについて、scoreごとのヒストグラム表示
fig, axes = plt.subplots(len(df_train["context"].unique())//3+1, 3, figsize=(17,150))

i = 0
j = 0
color_names = {}

for ctext in sorted(list(df_train["context"].unique())):
    df = df_train[df_train["context"]==str(ctext)]
    
    if j == 3:
        i += 1
        j = 0
    else:
        pass
    
    axes[i, j].set_title(str(ctext))
    axes[i, j].bar(df["score"].value_counts().sort_values(ascending=True).index,
                   df["score"].value_counts().sort_values(ascending=True),
                   tick_label = df["score"].value_counts().sort_values(ascending=True).index,width=0.2)
    j += 1

基本的には0.25,0.50が最頻値となっている。\
一部のsectionにおいてはscore0.75,1.0が一つもない

In [ ]:
# score1.0について中身を確認
df_train[df_train["score"]==1.0][30:50]

score1.0のanchorとtargetはほとんど同じ単語が使われている。\
単語の順序が入れ替わっていたり、単語が進行形であるか否かの違いでしかないものがほとんど

In [ ]:
# score0.75について中身を確認
df_train[df_train["score"]==0.75][30:50]

2つの単語の内1つがanchorとtargetで被っているものが多い\
ただし、acrylate groupsとprop 2 enoatesのようにまったく被っていないものもある\
またadapt for learningとadaptive learningのようにほぼ同じ語句でもscore0.75のものがある

In [ ]:
# score0.00について中身を確認
df_train[df_train["score"]==0.00][30:50]

score0.75の場合と比べると、anchorとtargetで全然違う単語が多い
absorbent propertiesとplastic propertiesのように1語被っているものもあるが、
そういう場合はpropertiesやinformationのようにどんな単語ともくっつけられそうなものが多い（？）

In [ ]:
# score0.00かつsection==Aについて中身を確認
df_train[(df_train["score"]==0.00) & (df_train["section"]=="A")][30:50]

In [ ]:
# score0.00かつsection==Cについて中身を確認
df_train[(df_train["score"]==0.00) & (df_train["section"]=="C")][30:50]

A（人間の必需品）はanchorとtargetがほとんど1語被りだが、C（化学及び治金学）はanchorとtargetがまったく被っていない

### 単語を語幹化して再度確認

In [ ]:
import re
import nltk
nltk.download('all')
from nltk import stem
stemmer = stem.LancasterStemmer()


In [ ]:
# 語幹化処理
anchor_stem_list = []
target_stem_list = []
anc_and_tars = []

for row in range(len(df_train)):
    anc = df_train["anchor"][row].split(" ")
    tar = df_train["target"][row].split(" ")
    
    anc_stem = [stemmer.stem(i) for i in anc]
    tar_stem = [stemmer.stem(i) for i in tar]
    
    anc_and_tars.append(len(set(anc_stem)&set(tar_stem)))
    anchor_stem_list.append(" ".join(anc_stem))
    target_stem_list.append(" ".join(tar_stem))
    
df_train["anc_and_tar"] = anc_and_tars
df_train["anchor_stem"] = anchor_stem_list
df_train["target_stem"] = target_stem_list

In [ ]:
# 語幹化後データ⇒anchor_stem,target_stem
# anc_and_tar ⇒anchor_stemとtarget_stemで同じ単語が使われている回数
df_train

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
df_train["anc_and_tar"].hist(by=df_train["score"])